<img src="https://cdn.comet.ml/img/notebook_logo.png">

# Introduction

[Gradio](https://www.gradio.app/) allows you to quickly create customizable UI components around your TensorFlow or PyTorch models, or even arbitrary Python functions. Mix and match components to support any combination of inputs and outputs. Our core library is free and open-source!

[Comet](https://www.comet.ml/site/data-scientists/?utm_campaign=gradio-integration&utm_medium=colab) is an MLOps Platform that is designed to help Data Scientists and Teams build better models faster! Comet provides tooling to track, Explain, Manage, and Monitor your models in a single place! It works with Jupyter Notebooks and Scripts and most importantly it's 100% free!

In this notebook, we will go over a few of the User Interfaces you can create for your models with Gradio and how to log them to Comet and view them with the Gradio Panel.          

# Setup

This integration enables you to access and interact with your Gradio Interface from within Comet.

To do so, search for “Gradio” in the Public section of the Comet Panels Gallery. Add it to your project or experiment, and run any of the example snippets below to get started.

**Note:** Your UI's will expire after 24 hours if you are not using a hosted version of Gradio. Find out more about hosting your UI's [here](https://www.gradio.app/sharing_your_app/).

## Install Dependencies

In [ ]:
%pip install -U comet_ml gradio altair torch torchvision "transformers<4.40.0" requests Pillow

In [ ]:
!apt install python3.10-venv

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  python3-pip-whl python3-setuptools-whl
The following NEW packages will be installed:
  python3-pip-whl python3-setuptools-whl python3.10-venv
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 2,473 kB of archives.
After this operation, 2,884 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 python3-pip-whl all 22.0.2+dfsg-1ubuntu0.4 [1,680 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 python3-setuptools-whl all 59.6.0-1.2ubuntu0.22.04.1 [788 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 python3.10-venv amd64 3.10.12-1~22.04.3 [5,716 B]
Fetched 2,473 kB in 1s (1,848 kB/s)
Selecting previously unselected package python3-pip-whl.
(Reading database ... 121925 files and directories currently installed.)
Pr

In [ ]:
!apt install git-all

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  apache2 apache2-bin apache2-data apache2-utils cvs cvsps git-cvs git-doc git-email git-gui
  git-mediawiki git-svn gitk gitweb libalgorithm-c3-perl libapr1 libaprutil1
  libaprutil1-dbd-sqlite3 libaprutil1-ldap libauthen-sasl-perl libb-hooks-endofscope-perl
  libb-hooks-op-check-perl libcgi-fast-perl libcgi-pm-perl libclass-c3-perl libclass-c3-xs-perl
  libclass-data-inheritable-perl libclass-inspector-perl libclass-method-modifiers-perl
  libclass-singleton-perl libclass-xsaccessor-perl libclone-perl libcommon-sense-perl
  libdata-dump-perl libdata-optlist-perl libdatetime-format-builder-perl
  libdatetime-format-iso8601-perl libdatetime-format-strptime-perl libdatetime-locale-perl
  libdatetime-perl libdatetime-timezone-perl libdbd-sqlite3-perl libdbi-perl
  libdevel-callchecker-perl libdevel-caller-perl libdevel-lexalias-perl libdeve

In [ ]:
!git init

hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/.git/


In [ ]:
!git branch -m COMET_GIT_DIRECTORY

## Initialize Comet and Gradio

In [ ]:
import comet_ml
import gradio as gr

comet_ml.init(project_name="comet-example-gradio-notebook")

COMET WARNING: config setting 'COMET_GIT_DIRECTORY' failed with value '/content/drive/MyDrive/COMET_GIT_DIRECTORY': invalid setting


In [ ]:
from comet_ml import Experiment
from comet_ml.integration.pytorch import log_model

experiment = Experiment(
  api_key="Uv1tHzG08ZGimSU2iJ6USMLNE",
  project_name="general",
  workspace="kilos11",
      )

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : slimy_nut_640
COMET INFO:     url                   : https://www.comet.com/kilos11/general/3ce28aca90aa45a9b6cf02cde2b90c21
COMET INFO:   Uploads:
COMET INFO:     environment details : 1
COMET INFO:     filename            : 1
COMET INFO:     installed packages  : 1
COMET INFO:     notebook            : 2
COMET INFO:     os packages         : 1
COMET INFO:     source_code         : 1
COMET INFO: 
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/kilos11/general/eb

# Interfaces for Image Models

Lets set up a some interfaces to test out Image Classification.

After running the example, you will see a link to a Comet Experiment with the logged UI. If you head over to the Panels tab in the Experiment, you will a Gradio Panel that you can use to view the UI.  

## Image Classification

In this example, we will build an interface to interact with an image classification model `resnet18`:


In [ ]:
import requests  # Import the requests library for making HTTP requests
import torch  # Import the PyTorch library for deep learning
from PIL import Image  # Import the Image module from the Python Imaging Library (PIL)
from torchvision import transforms  # Import the transforms module from PyTorch's torchvision package

# Download the pre-trained ResNet18 model and a sample image
torch.hub.download_url_to_file(
    "https://github.com/pytorch/hub/raw/master/images/dog.jpg", "dog.jpg"
)

# Load the pre-trained ResNet18 model and set it to evaluation mode
model = torch.hub.load(
    "pytorch/vision:v0.6.0",
    "resnet18",
    pretrained=True
).eval()

# Download human-readable labels for ImageNet
response = requests.get("https://git.io/JJkYN")
# Split the response text into a list of labels
labels = response.text.split("\n")

def predict(inp):
    """
    Predict the class probabilities for a given input image.

    Args:
    inp (numpy.ndarray): The input image as a numpy array.

    Returns:
    dict: A dictionary mapping class labels to their corresponding probabilities.
    """
    # Convert the input array to a PIL Image
    inp = Image.fromarray(inp.astype("uint8"), "RGB")
    # Convert the PIL Image to a PyTorch tensor
    inp = transforms.ToTensor()(inp).unsqueeze(0)

    # Disable gradient computation for inference
    with torch.no_grad():
        # Get the class probabilities
        prediction = torch.nn.functional.softmax(model(inp)[0], dim=0)
    # Convert to a dictionary
    return {labels[i]: float(prediction[i]) for i in range(1000)}

## Set up the Gradio interface
# Define the input component as an image
inputs = gr.Image()
# Define the output component as a label with top 3 classes
outputs = gr.Label(num_top_classes=3)

# Create the Gradio interface
io = gr.Interface(
    fn=predict,
    inputs=inputs,
    outputs=outputs,
    examples=["dog.jpg"]
)

# Launch the interface in a new window and share it
io.launch(inline=False, share=True)

# Set up the CometML experiment
experiment = comet_ml.Experiment()
# Add a tag to the experiment
experiment.add_tag("image-classifier")

# Integrate the Gradio interface with CometML
io.integrate(comet_ml=experiment)

100%|██████████| 646k/646k [00:00<00:00, 13.6MB/s]
Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.6.0
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://53cc1d4f3e9c2d2176.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : mysterious_dog_5274
COMET INFO:     url                   : https://www.comet.com/kilos11/general/eb485b2b298c40cda1821afd73a9c85e
COMET INFO:   Uploads:
COMET INFO:     environment details : 1
COMET INFO:     filename            : 1
COMET INFO:     git metadata        : 1
COMET INFO:     installed packages  : 1
COMET INFO:     notebook            : 2
COMET INFO:     os packages         : 1
COMET INFO:     source_code         : 1
COMET INFO: 
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on come

In [ ]:
!git clone https://github.com/kilos11/COMET_GIT_DIRECTORY /content/COMET_DIRECTORY

Cloning into '/content/COMET_DIRECTORY'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 4 (delta 0), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (4/4), 4.77 KiB | 4.77 MiB/s, done.


In [ ]:
import os
os.environ['COMET_GIT_DIRECTORY'] = '/content/your-git-repo'

# Interfaces for NLP Models

Lets build some interfaces to interact with Transformers.

#**Text Generation**#



##*1. **Import statements**: These lines import the necessary libraries and modules for the code to work.*
##*2. **Device selection**: These lines check if a CUDA-enabled GPU is available and set the device accordingly.*
##*3. **Model loading**: This line loads the pre-trained GPT-2 model for causal language modeling.*
##*4. **Model configuration**: These lines set the model configuration for text generation, including sampling parameters, maximum length, and n-gram repetition avoidance.*
##*5. **Tokenizer loading**: This line loads the tokenizer for the pre-trained model.*
##*6. **Text generation function**: This function takes an input prompt, tokenizes it, generates text using the pre-trained model, and returns the generated text with some post-processing.*
##*7. **Setting up the Gradio interface**: These lines define the input and output components for the Gradio interface and create the interface itself.*
##*8. **Launching the Gradio interface**: This line launches the Gradio interface in a new window and shares it for others to access.*
##*9. **Setting up the CometML experiment**: These lines set up a CometML experiment and add a tag to it.*
##*10. **Integrating Gradio with CometML**: This line integrates the Gradio interface with the CometML experiment for tracking and logging purposes.*



In [ ]:
import torch  # Import the PyTorch library for deep learning
# Import the required modules from the Hugging Face Transformers library
from transformers import AutoModelForCausalLM, AutoTokenizer

# Check if a CUDA-enabled GPU is available, and set the device accordingly
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

# Set the name of the pre-trained model to use
MODEL_NAME = "gpt2"

# Load the pre-trained GPT-2 model for causal language modeling
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)

## Set the model configuration for text generation
# Set the model as a decoder
model.config.is_decoder = True
model.config.task_specific_params["text-generation"] = {
    # Enable sampling for text generation
    "do_sample": True,
    # Set the maximum length of the generated text
    "max_length": 50,
    # Set the temperature for sampling (higher values lead to more diverse outputs)
    "temperature": 0.7,
    # Consider the top-k tokens for sampling
    "top_k": 50,
    # Avoid repeating n-grams of this size
    "no_repeat_ngram_size": 2,
}

# Move the model to the selected device (CPU or GPU)
model = model.to(device)
# Load the tokenizer for the pre-trained model
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def generate_text(inp):
    """
    Generate text based on the given input prompt.

    Args:
    inp (str): The input prompt for text generation.

    Returns:
    str: The generated text.
    """
    # Tokenize the input prompt
    input_ids = tokenizer.encode(inp, return_tensors="pt")

    beam_output = model.generate(
        # Move the input tensor to the selected device
        input_ids.to(device),
        # Set the maximum length of the generated text
        max_length=100,
        # Use beam search with 5 beams
        num_beams=5,
        # Avoid repeating n-grams of size 2
        no_repeat_ngram_size=2,
        # Stop generation as soon as the end-of-sequence token is generated
        early_stopping=True,
    )

    output = tokenizer.decode(
        # Decode the generated output
        beam_output[0], skip_special_tokens=True, clean_up_tokenization_spaces=True
    )
    # Clean up the output by removing the last incomplete sentence
    return ".".join(output.split(".")[:-1]) + "."


## Set up the Gradio interface
# Define the input component as a text box
input_text = gr.Textbox(label="Input Prompt")
# Define the output component as a text box
output_text = gr.Textbox(label="Generated Output")

# Create the Gradio interface
io = gr.Interface(
    generate_text,
    inputs=input_text,
    outputs=output_text,
    examples=[["The detective looked at the room full of suspects and said, "]],
)

# Launch the interface in a new window and share it
io.launch(inline = False, share=True)

# Set up the CometML experiment
experiment = comet_ml.Experiment()
# Add a tag to the experiment
experiment.add_tag("text-generation")

# Integrate the Gradio interface with CometML
io.integrate(comet_ml=experiment)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()


COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.


Running on public URL: https://20b15f12369e89c1a6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


COMET INFO: Experiment is live on comet.com https://www.comet.com/kilos11/comet-example-gradio-notebook/1bbab778e76f4c6aad84bcfb1509b2aa

COMET ERROR: Couldn't log ipython notebook
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : notable_proton_1669
COMET INFO:     url                   : https://www.comet.com/kilos11/comet-example-gradio-notebook/1bbab778e76f4c6aad84bcfb1509b2aa
COMET INFO:   Others:
COMET INFO:     Created from : Gradio
COMET INFO:   Uploads:
COMET INFO:     environment details : 1
COMET INFO:     filename            : 1
COMET INFO:     git metadata        : 1
COMET INFO:     installed packages  : 1
COMET INFO:     notebook            : 1
COMET INFO:     os packages         : 1
C

## Question Answering

Let's build a UI for question answering with DistilBert.

In [ ]:
from transformers import (
    AutoTokenizer,
    AutoModelForQuestionAnswering,
    pipeline,
)

if torch.cuda.is_available():
    device = "cuda:0"
else:
    device = "cpu"

do_lower_case = True
model_version = "distilbert-base-uncased-distilled-squad"

tokenizer = AutoTokenizer.from_pretrained(model_version, do_lower_case=do_lower_case)
tokenizer.pad_token = "[PAD]"
model = AutoModelForQuestionAnswering.from_pretrained(
    model_version, output_attentions=True, pad_token_id=tokenizer.eos_token_id
)

qa = pipeline("question-answering", model=model, tokenizer=tokenizer, device=device)


def qa_func(context, question):
    prediction = qa(question=question, context=context)
    answer = prediction["answer"]

    return answer


io = gr.Interface(
    qa_func,
    inputs=[
        gr.Textbox(lines=7, label="Context"),
        gr.Textbox(label="Question"),
    ],
    outputs=[gr.Textbox(label="Answer")],
    examples=[
        [
            """A Moon landing is the arrival of a spacecraft on the surface of the Moon.
    This includes both crewed and robotic missions. The first human-made object to touch the Moon was the Soviet Union's Luna 2, on 13 September 1959.
    The United States' Apollo 11 was the first crewed mission to land on the Moon, on 20 July 1969.
    There were six crewed U.S. landings between 1969 and 1972, and numerous uncrewed landings, with no soft landings happening between 22 August 1976 and 14 December 2013.
    """,
            "What year did the first crewed mission land on the moon?",
        ]
    ],
)
io.launch(inline=False, share=True)

experiment = comet_ml.Experiment()
experiment.add_tag("question-answering")

io.integrate(comet_ml=experiment)